In [1]:
import langchain
import os
import warnings
warnings.filterwarnings('ignore')

# 환경변수 세팅

os.environ['OPENAI_API_KEY'] = '8c91c5c7058847f3820f9d7c12bbb1fb'
os.environ['AZURE_OPENAI_ENDPOINT'] = 'https://sktfly-ai.openai.azure.com/'
os.environ['OPENAI_API_TYPE'] = 'azure'
os.environ['OPENAI_API_VERSION'] = '2023-05-15'

In [7]:
from langchain.embeddings import AzureOpenAIEmbeddings

embedding_model = AzureOpenAIEmbeddings(
   # deployment_name = 'dev-text-embedding-ada-002',
    model = 'dev-text-embedding-ada-002' # 곧 deploy_name으로 변경될 예정
)

In [25]:
embedding_result = embedding_model.embed_documents(
    [
        '안녕하세요',
        '제 이름은 홍길동입니다.',
        '이름은 무엇인가?',
        '랭체인은 유용합니다.',
        'Hello World!'
    ]
)


list(map(len, embedding_result))

[1536, 1536, 1536, 1536, 1536]

In [26]:
embedding_query_q = embedding_model.embed_query('이 대화에서 언급된 이름은 무엇입니까?')
embedding_query_a = embedding_model.embed_query('이 대화에서 언급된 이름은 홍길동입니다.')

len(embedding_query_q), len(embedding_query_a)

(1536, 1536)

In [14]:
import numpy as np

cos_sim = lambda a, b: np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [28]:
cos_sim(embedding_model.embed_query('너 뭐햐냐'),embedding_model.embed_query('에헤이 이거 조졌네'))

0.8347529885108067

In [27]:
print(cos_sim(embedding_query_q, embedding_query_a))
print(cos_sim(embedding_query_q, embedding_result[1]))
print(cos_sim(embedding_query_q, embedding_result[3]))

0.9017447932333962
0.8504563306662587
0.7762442819388619


## Hugginface 모델 사용하기

In [29]:
!pip install sentence-transformers -q

In [43]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = 'BAAI/bge-small-en'
model_kwargs = {'device': 'mps'}
encode_kwargs = {'normalize_embeddings': True} # 일반화된 임베딩을 사용할 것인지 여부

hf = HuggingFaceBgeEmbeddings(
    model_name = model_name,
    model_kwargs = model_kwargs,
    encode_kwargs = encode_kwargs
)

hf



HuggingFaceBgeEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': True}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False})
  (2): Normalize()
), model_name='BAAI/bge-small-en', cache_folder=None, model_kwargs={'device': 'mps'}, encode_kwargs={'normalize_embeddings': True}, query_instruction='Represent this question for searching relevant passages: ')

In [44]:
embedding_result = hf.embed_documents(
    [
        'today is monday',
        'weather is nice today',
        'what is the problem?',
        'langchain is useful',
        'Hello World!',
        'my name is morris'
    ]
)

print(list(map(len, embedding_result)))


[384, 384, 384, 384, 384, 384]


In [45]:
BGE_query_q = hf.embed_query('Hello? who is this?')
BGE_query_a = hf.embed_query('hi this is harrison')

print(len(BGE_query_q), len(BGE_query_a))
print(cos_sim(BGE_query_q, BGE_query_a))

384 384
0.8522539761253092


In [46]:
BGE_query_q = hf.embed_query('Hello? who is this?')
BGE_query_a = hf.embed_query('I am an idiot')

print(len(BGE_query_q), len(BGE_query_a))
print(cos_sim(BGE_query_q, BGE_query_a))

384 384
0.7677968113692798


In [48]:
model_name = 'jhgan/ko-sbert-nli'
model_kwargs = {'device': 'mps'}
encode_kwargs = {'normalize_embeddings': True} # 일반화된 임베딩을 사용할 것인지 여부

hf = HuggingFaceBgeEmbeddings(
    model_name = model_name,
    model_kwargs = model_kwargs,
    encode_kwargs = encode_kwargs
)

config_sentence_transformers.json: 100%|██████████| 123/123 [00:00<00:00, 947kB/s]
README.md: 100%|██████████| 4.46k/4.46k [00:00<00:00, 6.39MB/s]
sentence_bert_config.json: 100%|██████████| 53.0/53.0 [00:00<00:00, 198kB/s]
config.json: 100%|██████████| 620/620 [00:00<00:00, 2.12MB/s]
pytorch_model.bin: 100%|██████████| 443M/443M [00:47<00:00, 9.29MB/s] 
tokenizer_config.json: 100%|██████████| 538/538 [00:00<00:00, 1.53MB/s]
vocab.txt: 100%|██████████| 248k/248k [00:00<00:00, 474kB/s]
tokenizer.json: 100%|██████████| 495k/495k [00:00<00:00, 928kB/s]
1_Pooling/config.json: 100%|██████████| 190/190 [00:00<00:00, 594kB/s]


In [49]:
sentence = [
    '안녕하세요',
    '제 이름은 홍길동입니다.',
    '이름은 무엇인가요?',
    '랭체인은 유용합니다.',
    '홍길동 아버지의 이름은 홍판서입니다.',
]

embedding_result = hf.embed_documents(sentence)

print(list(map(len, embedding_result)))

[768, 768, 768, 768, 768]


In [59]:
q = hf.embed_query('홍길동은 아버지를 아버지라 부르지 못했습니다. 홍길동 아버지의 이름은 무엇인가요?')
a = hf.embed_query('홍길동은 어머니를 어머니라 부르지 못했습니다. 홍길동 어머니의 이름은 무엇인가요?')

print(cos_sim(q, a))

0.8996751103853661
